In [41]:
######################
## Método de Jacobi ##
######################

import numpy as np
import matplotlib.pyplot as plt

# Parametros
N = 60
M = N
deltar = 1. / (N + 1)
deltat = M // 10

# Crea arreglo de coordenadas
i_idx, j_idx = np.meshgrid(np.arange(1, N + 1), np.arange(1, N + 1))
x_vals = np.arange(N) * deltar

# Define rho y fija la solución teórica
sin_i = np.sin(2 * np.pi * i_idx * deltar)
sin_j = np.sin(2 * np.pi * j_idx * deltar)
rho = -sin_i * sin_j
v1_theo = 0.5 * sin_i * sin_j / (2 * np.pi)**2

# Inicializa arreglos
v = np.zeros((N + 2, N + 2))
v1 = np.zeros((N, N))
err_j = np.empty(M)
its = np.arange(1, M + 1)

count = 0
for k in range(M):
    # Grafica snapshots cada deltat iteraciones
    if k % deltat == 0:
        vx = v1[:, N // 4]
        vx_theo = v1_theo[:, N // 4]

        plt.figure(figsize=(10, 10))
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        plt.ylim(-0.015, 0.015)
        plt.plot(x_vals, vx, color='blue', label='Numerical')
        plt.plot(x_vals, vx_theo, color='red', label='Theoretical')
        plt.legend()

        filename = f'test{count:03d}.png'
        plt.savefig(filename)
        plt.close()
        count += 1
        print('count=',count)

    # aplica método de Jacobi
    v1 = 0.25 * (v[2:, 1:-1] + v[:-2, 1:-1] + v[1:-1, 2:] + v[1:-1, :-2] - rho * deltar**2)

    # Error
    diff = v1 - v[1:-1, 1:-1]
    err_j[k] = np.sqrt(np.mean(diff**2))

    # actualiza v para la próxima iteración
    v[1:-1, 1:-1] = v1
    # el no hacer nada con los bordes asegura que estos seguiran valiendo cero

# Plot error convergence
plt.figure(figsize=(10, 10))
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.yscale('log')
plt.plot(its, err_j, color='blue', label='Jacobi')
plt.legend(loc='upper right', frameon=False)
plt.savefig('err_jacobi.png')
plt.close()

count= 1
count= 2
count= 3
count= 4
count= 5
count= 6
count= 7
count= 8
count= 9
count= 10


In [42]:
############################
## Método de Gauss-Seidel ##
############################

import numpy as np
import matplotlib.pyplot as plt

v = np.zeros((N + 2, N + 2))
v_old = np.zeros((N, N))
err_gs = np.empty(M)

# loop principal
count = 0
for k in range(M):
    if k % deltat == 0:
        vx = v[1:N+1, N // 4 + 1]
        plt.figure(figsize=(10, 10))
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        plt.ylim(-0.015, 0.015)
        plt.plot(x_vals, vx, color='blue', label='Gauss-Seidel')
        plt.plot(x_vals, vx_theo, color='red', label='Theoretical')
        plt.legend()
        plt.savefig(f'test{count:03d}.png')
        plt.close()
        print(f'count={count}')
        count += 1

    # Guarda valores previos para calcular errores
    v_old[:, :] = v[1:-1, 1:-1]

    # Método de Gauss-Seidel (en el lugar)
    for i in range(1, N + 1):
        for j in range(1, N + 1):
            v[i, j] = 0.25 * (v[i+1, j] + v[i-1, j] + v[i, j+1] + v[i, j-1]) \
                      - 0.25 * rho[i-1, j-1] * deltar**2

    # Error
    err = np.sqrt(np.mean((v_old - v[1:-1, 1:-1])**2))
    err_gs[k] = err

# Plot error
plt.figure(figsize=(10, 10))
plt.xlabel('Iteracion')
plt.ylabel('Error')
plt.yscale("log")
plt.plot(its, err_gs, color='red', label='Gauss-Seidel')
plt.plot(its, err_j, color='blue', label='Jacobi')
plt.legend(loc='upper right', frameon=False)
plt.savefig('err_gs.png')
plt.close()

count=0
count=1
count=2
count=3
count=4
count=5
count=6
count=7
count=8
count=9


In [43]:
######################
##    Método SOR    ##
######################

import numpy as np
import matplotlib.pyplot as plt

w = 2. / (1 + np.pi / N)

# Arreglos
v = np.zeros((N + 2, N + 2))
err_sor = np.empty(M)

count = 0
for k in range(M):
    # Snapshot de la solucion cada deltat iteraciones
    if k % deltat == 0:
        vx[:] = v[1:-1, N // 4 + 1]
        print(f'count={count}')
        plt.figure(figsize=(10, 10))
        plt.xlabel('$x$')
        plt.ylabel('$y$')
        plt.ylim(-0.015, 0.015)
        plt.plot(x_vals, vx, color='blue', label='SOR')
        plt.plot(x_vals, vx_theo, color='red', label='Theoretical')
        plt.legend()
        plt.savefig(f'test{count:03d}.png')
        plt.close()
        count += 1
        
    # Método SOR y cálculo de error
    v_old[:,:] = v[1:-1, 1:-1]#.copy()
    for i in range(1, N + 1):
        for j in range(1, N + 1):
            resid = (v[i + 1, j] + v[i - 1, j] + v[i, j + 1] + v[i, j - 1] - 4. * v[i, j]) \
                    - rho[i - 1, j - 1] * deltar ** 2
            v[i, j] += 0.25 * w * resid

    diff = v[1:-1, 1:-1] - v_old
    err_sor[k] = np.sqrt(np.mean(diff ** 2))

# Graficamos curvas de error
plt.figure(figsize=(10, 10))
plt.xlabel('Iteration')
plt.ylabel('Error')
plt.yscale('log')

plt.plot(its, err_j, color='blue', label='Jacobi')
plt.plot(its, err_gs, color='red', label='Gauss-Seidel')
plt.plot(its, err_sor, color='green', label='SOR')

plt.legend(loc='upper right', frameon=False)
plt.savefig('err_sor.png')
plt.close()

count=0
count=1
count=2
count=3
count=4
count=5
count=6
count=7
count=8
count=9
